In [0]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiLine", "true") \
    .option("quote", '"') \
    .option("escape", '"') \
    .csv("gs://bucket-autoai/recalls.csv")

In [0]:
pdf = df.toPandas()

In [0]:
pdf.head()

In [0]:
pdf.info()

In [0]:
pdf.isnull().sum()

In [0]:
pdf.dropna(subset=['parkIt'], inplace=True)
pdf.dropna(subset=['parkOutSide'], inplace=True)
pdf.dropna(subset=['Notes'], inplace=True)
pdf.dropna(subset=['overTheAirUpdate'], inplace=True)


In [0]:
pdf.duplicated().sum()

In [0]:
obj_cols = pdf.select_dtypes(include=["object"]).columns
pdf[obj_cols] = pdf[obj_cols].astype("string")

In [0]:
string_cols = pdf.select_dtypes(include=["string"]).columns

for c in string_cols:
    pdf[c] = (
        pdf[c]
        .fillna("")
        .str.lower()
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.replace(
            r"[^a-z0-9àâäçéèêëîïôöùûüÿñæœ\s\.\,\;\:\?\!\'\-\_/]",
            " ",
            regex=True,
        )
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
    )


In [0]:
text_cols = ["Summary", "Consequence", "Remedy", "Notes"]

pdf["text_raw"] = (
    pdf[text_cols]
    .fillna("")
    .astype(str)
    .agg(" ".join, axis=1)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)


In [0]:
pdf["nb_characters"] = pdf["text_raw"].str.len()
pdf["nb_words"] = pdf["text_raw"].str.split().str.len()


In [0]:
pdf.head()